In [1]:
print("Hello World")

Hello World


In [2]:
!pip install stable_baselines3[extra]

In [3]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [4]:
env=gym.make('CartPole-v1', render_mode="human")

In [5]:
episodes=5

for episode in range(1, episodes+1):
    state=env.reset() # We get the initial set of the environment
    done=False
    score=0

    while not done:
        env.render() # View the graphical visualisation of the environment 
        action=env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        done=terminated or truncated
        score+=reward
        
    print(f"Episode: {episode}, Score: {score}")
env.close()

Episode: 1, Score: 17.0
Episode: 2, Score: 31.0
Episode: 3, Score: 14.0
Episode: 4, Score: 9.0
Episode: 5, Score: 17.0


In [6]:
'''
Two actions are there:
0. Push the cart to the left
1. Push the cart to the right
'''

env.action_space

Discrete(2)

In [7]:
env.action_space.sample() # Action space example

np.int64(1)

In [8]:
'''
There are 4 observations given along with their min and max value
NUM.      Observation                 MIN                     MAX
0.        Cart Position               -4.8                    4.8
1.        Cart Velocity               -INF                    INF
2.        Pole Angle                  -0.418 rad (-24 deg)    0.418 rad (24 deg)
3.        Pole Angular Velocity       -INF                    INF
'''

env.observation_space

Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)

In [9]:
env.observation_space.sample() # Observation space example

array([-0.49791777,  0.01082975,  0.05925753,  0.49490732], dtype=float32)

In [10]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


In [10]:
# Make your Directories
log_path = os.path.join('Training', 'Logs')

In [11]:
log_path

'Training\\Logs'

In [22]:
env=gym.make('CartPole-v1', render_mode="human")
env=DummyVecEnv([lambda: env])
# Multi layer Perceptron policy i.e, using deep NN policy
model=PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path) # We can use tensor-board to visualise metrics to evaluate

Using cpu device


In [23]:
model.learn(total_timesteps=20000)
# env.close()

Logging to Training\Logs\PPO_9
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 43   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 89          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009255767 |
|    clip_fraction        | 0.0998      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00112    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.41        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0155     |
|    value_loss           | 57.9        |
-----------------------------------------
---

In [24]:
PPO_Path=os.path.join('Training', 'Saved Models', 'PPO_Model_CartPole')

In [25]:
model.save(PPO_Path)

In [26]:
del model

In [27]:
PPO_Path

'Training\\Saved Models\\PPO_Model_CartPole'

In [28]:
model=PPO.load(PPO_Path, env)

In [30]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\ProgramData\miniconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(np.float64(500.0), np.float64(0.0))

In [31]:
env.close()

In [34]:
env = gym.make("CartPole-v1", render_mode="human")
obs, info = env.reset()

In [35]:
action, _=model.predict(obs)

In [37]:
episodes=5

for episode in range(1, episodes+1):
    obs, info=env.reset()
    done=False
    score=0

    while not done:
        env.render() # View the graphical visualisation of the environment 
        action, _=model.predict(obs) # Here we are using model now
        obs, reward, terminated, truncated, info = env.step(action)
        done=terminated or truncated
        score+=reward
        
    print(f"Episode: {episode}, Score: {score}")
env.close()

Episode: 1, Score: 476.0
Episode: 2, Score: 500.0
Episode: 3, Score: 324.0
Episode: 4, Score: 500.0
Episode: 5, Score: 291.0


In [12]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [13]:
save_path=os.path.join('Training', 'Saved Models')

In [14]:
train_env=gym.make("CartPole-v1")

In [15]:
stop_callback=StopTrainingOnRewardThreshold(reward_threshold=500, verbose=1)
eval_callback=EvalCallback(train_env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path=save_path, verbose=1)

In [56]:
model=PPO('MlpPolicy', train_env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [57]:
model.learn(total_timesteps=20500, callback=eval_callback)

Logging to Training\Logs\PPO_10
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22       |
|    ep_rew_mean     | 22       |
| time/              |          |
|    fps             | 668      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26          |
|    ep_rew_mean          | 26          |
| time/                   |             |
|    fps                  | 385         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008928662 |
|    clip_fraction        | 0.0978      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000201   

# Change Policies

In [16]:
net_arch=[dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [60]:
model=PPO('MlpPolicy', train_env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\ProgramData\miniconda3\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [61]:
model.learn(total_timesteps=20500, callback=eval_callback)

Logging to Training\Logs\PPO_11
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.1     |
|    ep_rew_mean     | 21.1     |
| time/              |          |
|    fps             | 580      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 27.7        |
|    ep_rew_mean          | 27.7        |
| time/                   |             |
|    fps                  | 371         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014114851 |
|    clip_fraction        | 0.204       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.0113     

# Tryinng alternate algorithms

In [5]:
from stable_baselines3 import DQN

In [18]:
model=DQN("MlpPolicy", train_env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14       |
|    ep_rew_mean      | 14       |
|    exploration_rate | 0.973    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1439     |
|    time_elapsed     | 0        |
|    total_timesteps  | 56       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 18.2     |
|    ep_rew_mean      | 18.2     |
|    exploration_rate | 0.931    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 534      |
|    time_elapsed     | 0        |
|    total_timesteps  | 146      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.51     |
|    n_updates        | 11       |
----------------------------------
----------------------------------
| rollout/            | 

In [20]:
model.save()

TypeError: BaseAlgorithm.save() missing 1 required positional argument: 'path'